In [ ]:
import pandas as pd
import os
import json
import random
from datetime import datetime, timedelta
from env import ori_path, mod_path, ntfy_nofi
pd.options.mode.chained_assignment = None
pd.set_option('display.max_columns',None)

In [ ]:
httpThr = pd.read_json(ori_path + "detect/httpthreat.json", precise_float=True, convert_dates=False, dtype=str)
httpThr.replace({"None": "null"}, inplace=True)

dnsCov = pd.read_json(ori_path + "detect/dnscorvertchannel.json", precise_float=True, convert_dates=False, dtype=str)
dnsCov.replace({"None": "null"}, inplace=True)

In [ ]:
httpThr_gen = pd.concat([httpThr] * 300, ignore_index=True)

In [ ]:
dnsCov_gen = pd.concat([dnsCov] * 300, ignore_index=True)

In [ ]:
httpThr_gen.srcAddr = "41.179.253.229"
dnsCov_gen.srcAddr = "41.179.253.229"

In [ ]:
def random_date(start, end):
    return start + timedelta(
        seconds=random.randint(0, int((end - start).total_seconds())))

In [ ]:
def random_datetime_with_nanoseconds(start, end):
    random_dt = random_date(start, end)
    # Generate a random nanoseconds value (0 to 999,999,999)
    nanoseconds = random.randint(0, 999_999_999)
    # Combine the datetime and the nanoseconds
    formatted_dt = f"{random_dt.isoformat()}.{str(nanoseconds).zfill(9)}+00:00"
    return formatted_dt

In [ ]:
start_date = datetime(2023, 7, 1)
end_date = datetime(2023, 7, 30)

In [ ]:
time_arr_http = []
for i in range(0, 300):
    formatted_datetime_ns = random_datetime_with_nanoseconds(start_date, end_date)
    time_arr_http.append(formatted_datetime_ns)
time_arr_http.sort()

In [ ]:
time_arr_dns = []
for i in range(0, 300):
    formatted_datetime_ns = random_datetime_with_nanoseconds(start_date, end_date)
    time_arr_dns.append(formatted_datetime_ns)
time_arr_dns.sort()

In [ ]:
time_df_http = pd.DataFrame(time_arr_http, columns=['time'])
time_df_dns = pd.DataFrame(time_arr_dns, columns=['time'])

In [ ]:
time_df_count_temp_http = pd.DataFrame()
time_df_count_temp_http['date'] = pd.to_datetime(time_df_http['time']).dt.date
time_df_count_http = time_df_count_temp_http.groupby('date').size().reset_index(name='http count')

In [ ]:
time_df_count_temp_dns = pd.DataFrame()
time_df_count_temp_dns['date'] = pd.to_datetime(time_df_dns['time']).dt.date
time_df_count_dns = time_df_count_temp_dns.groupby('date').size().reset_index(name='dns count')

In [ ]:
merged_count = pd.merge(time_df_count_http, time_df_count_dns, on='date')
merged_count['per day count'] = merged_count['http count'] + merged_count['dns count']
merged_count

In [ ]:
httpThr_gen.time = time_df_http
dnsCov_gen.time = time_df_dns

In [ ]:
httpThr_gen

In [ ]:
dnsCov_gen

In [ ]:
# output of csv
httpThr_gen.to_csv(mod_path + "detect/httpthreat.csv", index=False, sep="\t")
httpThr_gen.to_csv(mod_path + "detect/dnscorvertchannel.csv", index=False, sep="\t")

In [ ]:
# output of json
dnsCov_gen.to_json(mod_path + "detect/httpthreat.json", orient='records', date_format='iso')
dnsCov_gen.to_json(mod_path + "detect/dnscorvertchannel.json", orient='records', date_format='iso')